<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter05/Detectron2_Chapter05_Custom_Trainer.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 05-Custom Trainer
## The dataset
Should execute the Notebook titled "Data Processing" first to process the dataset or run the following code to download the processed dataset from GitHub repository of the book.


In [ ]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

## Train models
Installation

In [ ]:
!python -m pip install \
'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-9tmlmvai
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-9tmlmvai
     |████████████████████████████████| 50 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 153 kB 73.2 MB/s 
     |████████████████████████████████| 1.5 MB 62.8 MB/s 
     |████████████████████████████████| 549 kB 75.0 MB/s 
     |████████████████████████████████| 117 kB 67.0 MB/s 
     |████████████████████████████████| 96 kB 7.7 MB/s 
     |████████████████████████████████| 182 kB 75.3 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=5458332 sha256=4c8336f4970cdd8d3842cfb5d716969236bc7d64d592bc54314195f42b0c4810
  Stored in directory: /tmp/pip-ephem-wheel-cache-3uvftnhv/wheels/19/ac/6

In [ ]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

In [ ]:
from detectron2.data.datasets import register_coco_instances

In [ ]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_train = name_ds + "_train"
name_ds_test = name_ds + "_test"
image_root_train = name_ds + "/train"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_train = name_ds + "/train/" + af
json_file_test = name_ds + "/test/" + af

In [ ]:
# Register datasets
## train dataset
register_coco_instances(
    name = name_ds_train,
    metadata = {},
    json_file = json_file_train,
    image_root = image_root_train
    )
## test dataset
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )


### Training configuration

In [ ]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
import pickle
output_dir = "output/object_detector_custom"
os.makedirs(output_dir, exist_ok=True)
output_cfg_path = os.path.join(output_dir, "cfg.pickle")
nc = 2 # negative and positive classes
device = "cuda"
# Select a model
config_file_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [ ]:
# Create a configuration file
cfg = get_cfg()
config_file = model_zoo.get_config_file(config_file_url)
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Download weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Set datasets
cfg.DATASETS.TRAIN = (name_ds_train,)
cfg.DATASETS.TEST = (name_ds_test,)
# Workers
cfg.DATALOADER.NUM_WORKERS = 2
# Images per batch
cfg.SOLVER.IMS_PER_BATCH = 8
# Learning rate
cfg.SOLVER.BASE_LR = 0.00025
# Iterations
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.CHECKPOINT_PERIOD = 500
# Classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = nc
cfg.MODEL.DEVICE = device
cfg.OUTPUT_DIR = output_dir

In [ ]:
# save configuration file for future use
with open(output_cfg_path, "wb") as f:
  pickle.dump(cfg, f, protocol = pickle.HIGHEST_PROTOCOL)

### Training

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class BrainTumorTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder == None:
      output_folder = cfg.OUTPUT_DIR
    else:
      output_folder = os.path.join(cfg.OUTPUT_DIR, output_folder)
      os.makedirs(output_folder, exist_ok=True)
    return COCOEvaluator(dataset_name, distributed=False, output_dir=output_folder)


In [ ]:
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.CHECKPOINT_PERIOD

In [ ]:
trainer = BrainTumorTrainer(cfg)
trainer.train()

[12/20 17:42:27 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/20 17:43:14 d2.utils.events]:  eta: 2:18:48  iter: 19  total_loss: 32.15  loss_cls: 27  loss_box_reg: 0.4694  loss_rpn_cls: 2.62  loss_rpn_loc: 1.636  time: 1.6090  data_time: 0.1009  lr: 4.9953e-06  max_mem: 9635M
[12/20 17:43:46 d2.utils.events]:  eta: 2:16:44  iter: 39  total_loss: 8.32  loss_cls: 7.292  loss_box_reg: 0.06313  loss_rpn_cls: 0.5198  loss_rpn_loc: 0.3532  time: 1.6172  data_time: 0.0740  lr: 9.9902e-06  max_mem: 9635M
[12/20 17:44:19 d2.utils.events]:  eta: 2:15:08  iter: 59  total_loss: 2.409  loss_cls: 2.116  loss_box_reg: 0.01285  loss_rpn_cls: 0.1515  loss_rpn_loc: 0.1332  time: 1.6173  data_time: 0.0727  lr: 1.4985e-05  max_mem: 9635M
[12/20 17:44:51 d2.utils.events]:  eta: 2:14:20  iter: 79  total_loss: 1.164  loss_cls: 0.9165  loss_box_reg: 0.007242  loss_rpn_cls: 0.1219  loss_rpn_loc: 0.09936  time: 1.6136  data_time: 0.0743  lr: 1.998e-05  max_mem: 9635M
[12/20 17:45:23 d2.utils.events]:  eta: 2:13:41  iter: 99  total_loss: 0.9689  loss_cls: 0.7265  loss_

In [ ]:
# workaround if the next cell brings locale error
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# remove the .pth file
!rm {cfg.OUTPUT_DIR}/*.pth
# zip
!zip -r {cfg.OUTPUT_DIR}.zip {cfg.OUTPUT_DIR}
from google.colab import files
files.download(cfg.OUTPUT_DIR+".zip")